<a href="https://colab.research.google.com/github/ricardogr07/trabajo-final-aprendizaje/blob/main/src/Analisis_ML_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de dataset

## Importar librerías

In [ ]:
# Correr esta celda unicamente si se hace desde Colab
%pip install pandas numpy ucimlrepo

!git clone https://github.com/ricardogr07/trabajo-final-aprendizaje.git
%cd trabajo-final-aprendizaje/src
from evaluation_functions import *

fatal: destination path 'trabajo-final-aprendizaje' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
/content/trabajo-final-aprendizaje/src


In [ ]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
# ....

## Carga y Preprocesamiento de Datos

En esta sección se carga el conjunto de datos de archivos que contiene información...

### Carga del dataset

In [ ]:
cleveland_data_file_path = '../dataset/cleveland.data'
hungarian_data_file_path = '../dataset/hungarian.data'
switzerland_data_file_path = '../dataset/switzerland.data'
va_data_file_path = '../dataset/va.data'

column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

# Cargar todos los datos y generar un solo dataframe
try:
    data_cleveland = pd.read_csv(cleveland_data_file_path, names = column_names)
    data_hungarian = pd.read_csv(hungarian_data_file_path, names = column_names)
    data_switzerland = pd.read_csv(switzerland_data_file_path, names = column_names)
    data_va = pd.read_csv(va_data_file_path, names = column_names)

    # Agregar location como parte de las variables
    data_cleveland['location'] = 0
    data_hungarian['location'] = 1
    data_switzerland['location'] = 2
    data_va['location'] = 3

    df = pd.concat([data_cleveland, data_hungarian, data_switzerland, data_va], ignore_index=True)

except Exception as e:
    print(f"Error reading the file: {e}")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       920 non-null    float64
 1   sex       920 non-null    float64
 2   cp        920 non-null    float64
 3   trestbps  920 non-null    object 
 4   chol      920 non-null    object 
 5   fbs       920 non-null    object 
 6   restecg   920 non-null    object 
 7   thalach   920 non-null    object 
 8   exang     920 non-null    object 
 9   oldpeak   920 non-null    object 
 10  slope     920 non-null    object 
 11  ca        920 non-null    object 
 12  thal      920 non-null    object 
 13  target    920 non-null    int64  
 14  location  920 non-null    int64  
dtypes: float64(3), int64(2), object(10)
memory usage: 107.9+ KB


### Exploración inicial del dataset

### Preprocesar Datos

## Análisis exploratorio de datos


## Selección de modelo usando técnicas de aprendizaje supervisado


### Selección de características


### Validación de balanceo de clases


### Selección de modelo supervisado


### Ajuste de hiperparámetros del modelo


### Evaluación del modelo seleccionado


## Análisis de clúster con aprendizaje no supervisado


### PCA


### Clustering
